# MTurk: Approve / Reject Assignments + Post-Task Completion Processing of HITs

In [ ]:
import boto3
import json
import pandas as pd
import ast
from simple_colors import *
from typing import Dict
import re
import pprint
import string
from pathlib import Path
import os

In [ ]:
from mturk_processing_utils import (
    programmatically_review_HITs,
    get_dataset, 
    get_hit_id_dict, 
    parse_batch, 
    approved_parsed_batch_2_dicts,
    view_assignment,
    mturk,
    get_completetion_progress
)

In [ ]:
PROJECT_ROOT_DIR = '/Users/nehasrikanth/Documents/paraphrase-nlu/'
dataset_name = ['atomic', 'snli', 'social']


raw_data_paths = {d: os.path.join(PROJECT_ROOT_DIR, f'raw-data/defeasible-nli/defeasible-{d}') for d in dataset_name}

In [ ]:
programmatically_review_HITs(
    get_hit_id_dict(
        'mturk_data/creation/atomic_dnli_annotation_examples_3.json',
    )[1], 
    get_dataset(raw_data_paths['atomic'], 'atomic')
)
